In [72]:
#!pip install -r requirements.txt

In [2]:
import pandas as pd

In [8]:
df = pd.read_csv('./data/lyrics.csv', sep=';', encoding="utf-8")
df.shape

(13405, 7)

1. Remoção dos títulos das músicas que contenham os caracteres '()' 

In [9]:
df['title'].replace("\((.*?)\)", '', regex=True, inplace=True)



2. Remoção das músicas duplicadas a partir do título

In [10]:
df['title'] = df['title'].str.lower()
df['title'] = df['title'].str.strip()

titles = [i for i in df['title']]

duplicates = [i for i in titles if titles.count(i) > 1]
print(f'Quantidade de duplicatas: {len(duplicates)}')

Quantidade de duplicatas: 3484


In [11]:
df = df[~df['title'].isin(duplicates)]
print(f'Quantidade de músicas após remoção das duplicatas: {df.id.count()}')

Quantidade de músicas após remoção das duplicatas: 9921


3. Remoção das músicas sem data

In [12]:
df = df[df['year'] > 0]

print(f'Quantidade de músicas com data: {df.id.count()}')

Quantidade de músicas com data: 8176


4. Remoção dos registros sem letras

In [13]:
df = df[~df['lyric'].isnull()]
print(f'Quantidade de registros com letras: {df.lyric.count()}')

Quantidade de registros com letras: 8133


In [20]:
df = pd.read_csv('./data/clean_lyrics.csv')

In [21]:
#removendo ruídos das letras
df['clean_lyric'] = df['lyric'].apply(clean_lyrics)
df['clean_lyric'] = df['clean_lyric'].apply(remove_words_with_reps)
df['id'].count()

8128

Foi identificado que algumas letras que apresentavam 
<br>
sujeiras foram removidas pelo processo de limpeza.
<br> 
Devido a isso foi necessário repetir a limpeza dos 
<br>
registros que estavam com as letras das músicas nulas.

In [22]:
#removendo letras nulas
df = df[~df['clean_lyric'].isnull()]
df['id'].count()

8128

In [23]:
#gerando decadas
df['decade'] = df['year'].apply(decade)
#df.tail()

In [24]:
# gerando colunas de palavras únicas
df['unique_words'] = df['clean_lyric'].apply(unique)
df['id'].count()

8128

In [25]:
# transformando letras em lista de palavras
df['words'] = df['clean_lyric'].apply(to_words)
df.head()

,id,title,artist,composer,album,year,lyric,decade,clean_lyric,unique_words,words
0,2360,João Teimoso,Marília Batista,Noel Rosa / Marilia Batista.,Poeta da Vila,1953,Tenho mais o que fazer Não discuto com teimoso...,50,discuto teimoso perder precioso viver joão tei...,"{nome, cismo, dorme, cara, fome, coroas, bebid...","[discuto, teimoso, perder, precioso, viver, jo..."
1,2361,Morena Sereia,Marília Batista,José Maria De Abreu 1936 / Noel Rosa.,Poeta da Vila,1953,Morena sereia Que à beira-mar não passeia Que ...,50,morena sereia beiramar passeia senta praia dei...,"{nome, cheia, nomes, beiramar, esperança, soub...","[morena, sereia, beiramar, passeia, senta, pra..."
2,2362,"Parabéns, Guanabara",Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,"Rio, querida Guanabara Eu sou gente também Ace...",50,rio querida guanabara gente aceito parabéns pa...,"{céu, apertar, adoram, noel, brasileiro, queri...","[rio, querida, guanabara, gente, aceito, parab..."
3,2363,O Bom Xaxador,Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,Depois do baião foi que veio o forró Depois do...,50,baião veio forró forró veio rojão pisada chão ...,"{forró, chão, rojão, mostrar, pisada, xaxar, v...","[baião, veio, forró, forró, veio, rojão, pisad..."
4,2364,Cheguei Agora,Jackson do Pandeiro,NaN,Jackson do Pandeiro - 1955,1955,Beira mar beira do rio Ê ê beira mar Cheguei a...,50,beira mar beira rio beira mar cheguei beira ma...,"{beira, coco, planta, candomblé, canta, olha, ...","[beira, mar, beira, rio, beira, mar, cheguei, ..."


In [17]:
#Passando os titulos para o padrão CamelCase
df['title'] = df['title'].str.title()

In [18]:
#armazenando
df.to_csv('./data/clean_lyrics.csv', index=False)

### Funções utilitárias

In [5]:
# função para remover palavras que contêm repetições
def remove_words_with_reps(text):
  words = text.split()
  for i, word in enumerate(words):
      reps = 0
      for j, character in enumerate(word):
        if j > 0 and character == word[j-1]:
            reps+=1
      if reps > 1:
        words.pop(i)
      
  return (' ').join(words)

In [6]:
#função para criar decadas
decade = lambda year: int(((((year - (year % 10)) / 10) % 10) * 10))

In [7]:
#função para retornar palavras em formato de lista
to_words = lambda text: text.split()

In [8]:
#função para retornar palavras unicas
unique = lambda text: set(text.split())

In [9]:
#carregando lista de acordes para remoção dentro das letras
chords_df = pd.read_csv('./data/chords.txt')
chords_dicty = chords_df.to_dict()
chords = set()

for i in chords_dicty['chord']:
    chords.add(chords_dicty['chord'][i].strip())
    
#função para remoção de acordes dentro de um texto
def remove_chords(text):
    txt_list = text.split()
    return (' ').join(
            word.strip()
            for word in txt_list
            if word.strip() not in chords
        )    

In [19]:
import spacy
from spacy.lang.pt.stop_words import STOP_WORDS

nlp = spacy.load('pt_core_news_lg')
#neg_words = ['não', 'nem', 'nenhuma', 'nenhum', 'nada', 'nunca']
noise = ['bê', 'ba', 'i', 'bó', 'bi',
        'xuá', 'iê', 'xié', 'ta', 'to',
        'ai', 'pro', 'bis', 'vou',
        'tô', 'tá', 'pra', 'mim',
        'aa', 'aai', 'aaii',
        'aah', 'ah', 'aaahh','ha', 
        'oo','oh', 'ooh', 'ê', 'e',
        'ho', 'eeh', 'eh', 
        'he', 'uhu', 'uhl',
        'uhuhu','uhuhh', 'uuhhh', 'huhuhu']

lyric_noise = set(noise)
stop_words = STOP_WORDS.union(lyric_noise)
stop_words = [word for word in stop_words]
#if word not in neg_words]

In [11]:
import string
import re

def clean_lyrics(text):
  txt = ''
  #removendo acordes
  txt = remove_chords(text)
  #removendo tags de repetição 2x, 4x, etc
  txt = txt.replace('[', '(').replace(']', ')')
  txt = re.sub(r'\([\d]x\)', '', txt)
  #removendo números
  txt = re.sub(r'[\d]', '', txt)
  #removendo palavras de orientação musical
  txt = re.sub(r'\((.*?)\)', '', txt)  
  #removendo pontuacao
  txt = txt.translate(str.maketrans("", "", string.punctuation))
  #caracteres em minusculo   
  txt = txt.lower()
  #remove stop words
  doc = nlp(txt)
  txt = ' '.join(token.text 
                  for token in doc 
                  if not token.is_digit
                  and token.text not in stop_words
                )
  
  return txt